In [25]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np

In [ ]:
def char_to_weight(character):
    if character == "x":
        return np.array([0, 1, 0, 0])
    if character == "y":
        return np.array([0, 0, 1, 0])
    if character == "z":
        return np.array([0, 0, 0, 1])
    if character == "i":
        return np.array([1, 0, 0, 0])
def mapping_single_pauliword_one_gate(character, instructors):
    # Map a single Pauliword to list
    # Example: xyy -> [0, 1, 0, 0][0,0,1,0][0,0,1,0] -- h --> [0, 0, 0, 1][0,0,-1,0][0,0,-1,0] = z(-y)(-y)
    from numpy import sin, cos, sqrt
    weights = char_to_weight(character)

    for gate, index, param in instructors:
        I, A, B, C = weights
        if gate == "h":
            weights = np.array([I, 0, 0, 1])
        if gate == "s":
            weights = np.array([I, 0, 1, 0])
        if gate == "t":
            weights = np.array([I, 1 / sqrt(2), 1 / sqrt(2), 0])
        if gate == "rx":
            weights = np.array([I, A, B * cos(param) - C * sin(param), B * sin(param) + C * cos(param)])
        if gate == "ry":
            weights = np.array([I, A * cos(param) + C * sin(param), B, C * cos(param) - A * sin(param)])
        if gate == "rz":
            weights = np.array([I, A * cos(param) - B * sin(param), B * cos(param) + A * sin(param), C])
    return weights

def construct_LUT(instructorss, num_qubits):
    lut = np.zeros((4, num_qubits))
    characters = ["i", "x", "y", "z"]
    for i in range(4):
        for j in range(num_qubits):
            lut[i][j] = mapping_single_pauliword_one_gate(characters[i], instructorss[j])
    return lut

In [67]:
class Instructor:
    def __init__(self, num_qubits):
        self.clusters = []
        self.cluster = []
        self.cluster_temp = []
        self.xcluster = []
        self.xcluster_temp = []
        self.xclusters = []
        self.instructors = []
        self.num_qubits = num_qubits
        self.barriers = [0] * self.num_qubits

    def append(self, gate, index, param=0):
        self.instructors.append([gate, index, param])

    def clustering(self):
        self.barriers = [0] * self.num_qubits
        while len(self.instructors) > 0:
            gate, index, _ = self.instructors[0]
            
            is_break = False
            if gate == "cx":
                self.barriers[index[0]] += 1
                self.barriers[index[1]] += 1
                if sum(self.barriers) >= self.num_qubits and np.all(self.barriers):
                    if len(self.instructors) > 1:
                        if self.instructors[1][0] != "cx":
                            is_break = True

                self.xcluster.append(self.instructors.pop(0))
            else:
                if self.barriers[index] == 0:
                    self.cluster.append(self.instructors.pop(0))
                else:
                    self.cluster_temp.append(self.instructors.pop(0))
            if is_break:
                if len(self.cluster) > 0:
                    self.clusters.append(self.cluster)
                self.instructors =  self.cluster_temp + self.instructors
                if len(self.xcluster) > 0:
                    self.xclusters.append(self.xcluster)
                self.cluster = []
                self.cluster_temp = []
                self.xcluster = []
                self.barriers = [0] * self.num_qubits
                is_break = False
        if len(self.cluster) > 0:
            self.clusters.append(self.cluster)
        if len(self.cluster_temp) > 0:
            self.clusters.append(self.cluster_temp)
        if len(self.xcluster) > 0:
            self.xclusters.append(self.xcluster)
        return 

ins = Instructor(4)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [68]:
ins.clusters

[[['h', 0, 0],
  ['h', 1, 0],
  ['h', 2, 0],
  ['h', 0, 0],
  ['h', 2, 0],
  ['h', 2, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0]],
 [['h', 0, 0],
  ['h', 1, 0],
  ['h', 0, 0],
  ['h', 1, 0],
  ['h', 2, 0],
  ['h', 0, 0],
  ['h', 2, 0]]]

In [69]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [1, 3], 0]]]

In [70]:
num_qubits = 3
ins = Instructor(num_qubits)
for i in range(num_qubits - 1):
    ins.append('cx', [i, i + 1])
ins.append('cx', [num_qubits - 1, 0])
for i in range(num_qubits):
    ins.append('h', i)
for i in range(num_qubits - 1):
    ins.append('cx', [i, i + 1])
ins.append('cx', [num_qubits - 1, 0])
for i in range(num_qubits):
    ins.append('h', i)
ins.clustering()

In [71]:
ins.clusters

[[['h', 0, 0], ['h', 1, 0], ['h', 2, 0]],
 [['h', 0, 0], ['h', 1, 0], ['h', 2, 0]]]

In [72]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [2, 0], 0]],
 [['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [2, 0], 0]]]